In [2]:
library(tidyverse)
library(lubridate)
library(corrplot)
library(tidyverse)
library(hash)
library(gprofiler2)
library(viridis)
library(VennDiagram)

set.seed(1992)

In [3]:
ipr_scan <- read.csv('/data/projects/julia.pratt/CS1_genomeSelection/scripts/vitvi.vcostv3.clean.pep.txt', sep='\t')

## need to filter for genes tested in DGE analysis
ipr_sub <- ipr_scan %>% 
  select(genename, e.value, ipr_accession) %>%
  filter(e.value < 1e-05) %>% 
  filter(ipr_accession != '') %>%
  select(genename, ipr_accession) %>%
  unique()

#also grab pfam
pfam_sub <- ipr_scan %>% 
  select(genename, analysis, signature_accession, signature_description, e.value, ipr_accession) %>%
  filter(e.value < 1e-05) %>% 
  filter(analysis == 'Pfam') %>%
  select(genename, signature_accession, signature_description, ipr_accession) %>%
  unique()

In [4]:
full_enrichment_test <- function(set_of_genes, ipr_sub, print_res=FALSE){
    ipr_testset <- ipr_sub[ipr_sub$genename %in% set_of_genes,]
    terms <- unique(ipr_testset$ipr_accession)
    print(str_interp("analyzing ${length(terms)} IPR accessions..."))
    enriched_terms <- c() 
    for (term in terms){
        genes_to_term_in_set <- nrow(ipr_testset[ipr_testset$ipr_accession == term,])
        genes_to_term_in_genome <- nrow(ipr_sub[ipr_sub$ipr_accession == term,])
        genes_not_term_in_genome <- 24769 - genes_to_term_in_genome
        size_of_set <- length(set_of_genes)
        test <- phyper(genes_to_term_in_set, genes_to_term_in_genome,
                       genes_not_term_in_genome, size_of_set, lower.tail=FALSE)
        
        if (test < 0.05/length(terms) & 
            genes_to_term_in_genome >= 10 & 
            genes_to_term_in_set >= 3){
            if (print_res == TRUE) {
                print(c(term, test, genes_to_term_in_set, genes_to_term_in_genome))
            }
            enriched_terms$et <- c(enriched_terms$et, term)
            enriched_terms$num <- c(enriched_terms$num, genes_to_term_in_set)
        }
    }
    return(enriched_terms)
}

full_pfam_enrichment_test <- function(set_of_genes, pfam_sub, print_res=FALSE){
    pfam_testset <- pfam_sub[pfam_sub$genename %in% set_of_genes,]
    terms <- unique(pfam_testset$signature_accession)
    print(str_interp("analyzing ${length(terms)} Pfam domains..."))
    enriched_terms <- c()
    for (term in terms){ 
        genes_to_term_in_set <- nrow(pfam_testset[pfam_testset$signature_accession == term,])
        genes_to_term_in_genome <- nrow(pfam_sub[pfam_sub$signature_accession == term,])
        genes_not_term_in_genome <- 24769 - genes_to_term_in_genome
        size_of_set <- length(set_of_genes)
        test <- phyper(genes_to_term_in_set, genes_to_term_in_genome,
                       genes_not_term_in_genome, size_of_set, lower.tail=FALSE)
        
        if (test < 0.05/length(terms) & genes_to_term_in_genome >= 10){
            if (print_res == TRUE){
                print(c(term, test, genes_to_term_in_set, genes_to_term_in_genome))
            }
            enriched_terms$et <- c(enriched_terms$et, term)
            enriched_terms$num <- c(enriched_terms$num, genes_to_term_in_set)
        }
        
    }
    
    return(enriched_terms)

}

In [11]:
## make whole analysis into a function
functional_enrichment <- function(file, ipr_scan, ipr_sub, pfam_sub){
    example_dataset <- read.csv(file)
    all_genes <- paste(example_dataset$X, '.t01', sep='')

    # get up regulated genes
    up_genes <- example_dataset %>%
      filter(padj < 0.05 & log2FoldChange > 0) %>%
      .[['X']] %>%
      paste('.t01', sep='')
    
    #get down regulated genes
    down_genes <- example_dataset %>%
      filter(padj < 0.05 & log2FoldChange < 0) %>%
      .[['X']] %>%
      paste('.t01', sep='')

    #filter for genes in DEG analysis
    ipr_sub_mtv <- ipr_sub %>%
      filter(genename %in% all_genes)

    pfam_sub_mtv <- pfam_sub %>%
      filter(genename %in% all_genes)
    
    #run ipr and pfam enrichments - Up regulation
    eset_up <- full_enrichment_test(up_genes, ipr_sub_mtv)
    eset_down <- full_enrichment_test(down_genes, ipr_sub_mtv)
    
    pset_up <- full_pfam_enrichment_test(up_genes, pfam_sub_mtv)
    pset_down <- full_pfam_enrichment_test(down_genes, pfam_sub_mtv)
    
    #make output tables
    ipr_up <- ipr_scan %>%
      select(ipr_accession, ipr_description) %>%
      unique() %>%
      filter(ipr_accession %in% eset_up$et)
    ipr_up<- merge(ipr_up, data.frame(eset_up), by.x="ipr_accession", by.y="et")
    
    ipr_down <- ipr_scan %>%
      select(ipr_accession, ipr_description) %>%
      unique() %>%
      filter(ipr_accession %in% eset_down$et)
    ipr_down<- merge(ipr_down, data.frame(eset_down), by.x="ipr_accession", by.y="et")
    
    pfam_up <- ipr_scan %>%
      filter(analysis == 'Pfam') %>% 
      select(signature_accession, signature_description) %>%
      unique() %>%
      filter(signature_accession %in% pset_up$et)
    pfam_up<- merge(pfam_up, data.frame(pset_up), by.x="signature_accession", by.y="et")
    
    pfam_down <- ipr_scan %>%
      filter(analysis == 'Pfam') %>% 
      select(signature_accession, signature_description) %>%
      unique() %>%
      filter(signature_accession %in% pset_down$et)
    pfam_down <- merge(pfam_down, data.frame(pset_down), by.x="signature_accession", by.y="et")
    
    output <- list()
    output[['full_eset_up']] <- eset_up
    output[['full_eset_down']] <- eset_down
    output[['full_pset_up']] <- pset_up
    output[['full_pset_down']] <- pset_down
    
    output[['pretty_eset_up']] <- ipr_up
    output[['pretty_eset_down']] <- ipr_down
    output[['pretty_pset_up']] <- pfam_up
    output[['pretty_pset_down']] <- pfam_down
    
    return(output)
}

In [12]:
tissues <- c('Leaf')
years <- c('2018')
phenos <- c('Anthesis', 'Veraison', 'Harvest')
rootstocks <- c('Ungrafted', '1103P', '3309C', 'SO4')
prefix <- '/data/projects/julia.pratt/CS1_genomeSelection/all_rootstock_comparisons/'
stats<- list()

# cycle through all the data
for (t in tissues){
    for (y in years){
        for (p in phenos){
            for (j in 2:length(rootstocks)){
                rs <- str_interp("Ungrafted-${rootstocks[j]}")
                fullpath <- paste(prefix, paste(t, y, p, rs, sep='_'), '.csv', sep='')
                fname <- paste(t,y,p,rs,sep="_")
                fnameformat <- paste(t,y,p,rs,sep=" ")
                d <- functional_enrichment(fullpath, ipr_scan=ipr_scan, ipr_sub=ipr_sub, pfam_sub=pfam_sub)
                stats[[fname]] <- d              
            }
        }
    }
}

[1] "analyzing 154 IPR accessions..."
[1] "analyzing 22 IPR accessions..."
[1] "analyzing 85 Pfam domains..."
[1] "analyzing 11 Pfam domains..."


ERROR: Error in fix.by(by.y, y): 'by' must specify a uniquely valid column


In [10]:
d

ipr_accession,ipr_description,num
IPR000217,Tubulin,11
IPR000490,Glycoside hydrolase family 17,19
IPR001087,GDSL lipase/esterase,27
IPR001563,"Peptidase S10, serine carboxypeptidase",23
IPR001715,Calponin homology domain,11
IPR001752,Kinesin motor domain,41
IPR003008,"Tubulin/FtsZ, GTPase domain",12
IPR003676,Small auxin-up RNA,33
IPR004159,Putative S-adenosyl-L-methionine-dependent methyltransferase,12
IPR004333,SBP domain,11


In [ ]:
for (n in names(stats)) {
    print(stats[[n]])
}

In [ ]:
plotvenn <- function(rs1103p, rs3309c, rsso4, fname, fnameformat){
    # change the name of the pdf saved based on what kind of venn is being generated
    fileprefix= "/data/projects/julia.pratt/CS1_genomeSelection/figs/go/venn/venn_pset_"
    f <- paste(fileprefix, fname, '.png', sep="")
    
    # the function takes in three lists
    # automatically counts how many are shared/private within those lists
    # most of this is just styling
    venn.diagram(
        x = list(rs1103p, rs3309c, rsso4),
        
        cex = 0.5,
        fontfamily = "sans",
        
        category.names = c("1103P", "3309C", "SO4"),
        cat.cex = 0.5,
        cat.default.pos = "outer",
        cat.dist = c(0.075, 0.075, 0.075),
        cat.fontfamily = "sans",
        
        main = fnameformat,
        main.cex = 0.6,
        main.fontfamily = "sans",
        
        col=c("#1b9e77", "#7570b3", "#e6ab02"),
        fill = c(alpha("#1b9e77",0.3), alpha("#7570b3",0.3), alpha("#e6ab02",0.3)),
        
        filename = f,
        output=TRUE, 
        resolution = 300,
        imagetype="png",
        margin=0.075,
        rotation = 1,
        height=700,
        width=700
    )
}

In [ ]:
plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]][["ipr_accession"]], 
         "leaf_anth_2018_eset_up", "Leaf Anthesis 2018 ESET Up")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]][["ipr_accession"]], 
         "leaf_ver_2018_eset_up", "Leaf Veraison 2018 ESET Up")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]][["ipr_accession"]], 
         "leaf_harv_2018_eset_up", "Leaf Harvest 2018 ESET Up")

plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]][["ipr_accession"]], 
         "leaf_anth_2018_eset_down", "Leaf Anthesis 2018 ESET Down")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]][["ipr_accession"]], 
         "leaf_ver_2018_eset_down", "Leaf Veraison 2018 ESET Down")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]][["ipr_accession"]], 
         "leaf_harv_2018_eset_down", "Leaf Harvest 2018 ESET Down")

In [ ]:
plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], 
         "leaf_anth_2018_pset_up", "Leaf Anthesis 2018 PSET Up")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], 
         "leaf_ver_2018_pset_up", "Leaf Veraison 2018 PSET Up")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], 
         "leaf_harv_2018_pset_up", "Leaf Harvest 2018 PSET Up")

plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], 
         "leaf_anth_2018_pset_down", "Leaf Anthesis 2018 PSET Down")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], 
         "leaf_ver_2018_pset_down", "Leaf Veraison 2018 PSET Down")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], 
         "leaf_harv_2018_pset_down", "Leaf Harvest 2018 PSET Down")

In [ ]:
pset.up.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]])
if(!is.null(pset.up.1103p)){
    d.pset.up.1103p = data.frame(gene=pset.up.1103p)
    d.pset.up.1103p = d.pset.up.1103p %>% arrange(gene)
    d.pset.up.1103p['color'] = viridis(length(pset.up.1103p))
    
    pal.ant <- d.pset.up.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P UP PSET')

    g.ver.u.1103p.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P UP PSET')

    g.har.u.1103p.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P UP PSET')
} else {
    print("no 1103p up")
}
pset.down.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]])
if(!is.null(pset.down.1103p)){
    d.pset.down.1103p = data.frame(gene=pset.down.1103p)
    d.pset.down.1103p = d.pset.down.1103p %>% arrange(gene)
    d.pset.down.1103p['color'] = viridis(length(pset.down.1103p))
    
    pal.ant <- d.pset.down.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P DOWN PSET')

    g.ver.u.1103p.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P DOWN PSET')

    g.har.u.1103p.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P DOWN PSET')
} else {
    print("no 1103p down")
}

pset.up.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]])
if(!is.null(pset.up.3309c)){
    d.pset.up.3309c = data.frame(gene=pset.up.3309c)
    d.pset.up.3309c = d.pset.up.3309c %>% arrange(gene)
    d.pset.up.3309c['color'] = viridis(length(pset.up.3309c))
    
    pal.ant <- d.pset.up.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C UP PSET')

    g.ver.u.3309c.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C UP PSET')

    g.har.u.3309c.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C UP PSET')
} else {
    print("no 3309c up")
}
pset.down.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]])
if(!is.null(pset.down.3309c)){
    d.pset.down.3309c = data.frame(gene=pset.down.3309c)
    d.pset.down.3309c = d.pset.down.3309c %>% arrange(gene)
    d.pset.down.3309c['color'] = viridis(length(pset.down.3309c))
    
    pal.ant <- d.pset.down.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C DOWN PSET')

    g.ver.u.3309c.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C DOWN PSET')

    g.har.u.3309c.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C DOWN PSET')
} else {
    print("no 3309c down")
}

pset.up.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]])
if(!is.null(pset.up.so4)){
    d.pset.up.so4 = data.frame(gene=pset.up.so4)
    d.pset.up.so4 = d.pset.up.so4 %>% arrange(gene)
    d.pset.up.so4['color'] = viridis(length(pset.up.so4))
    
    pal.ant <- d.pset.up.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 UP PSET')

    g.ver.u.so4.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 UP PSET')

    g.har.u.so4.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 UP PSET')
} else {
    print("no so4 up")
}
pset.down.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]])
if(!is.null(pset.down.so4)){
    d.pset.down.so4 = data.frame(gene=pset.down.so4)
    d.pset.down.so4 = d.pset.down.so4 %>% arrange(gene)
    d.pset.down.so4['color'] = viridis(length(pset.down.so4))
    
    pal.ant <- d.pset.down.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 DOWN PSET')

    g.ver.u.so4.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 DOWN PSET')

    g.har.u.so4.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 DOWN PSET')
} else {
    print("no so4 down")
}

In [ ]:
eset.up.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]])
if(!is.null(eset.up.1103p)){
    d.eset.up.1103p = data.frame(gene=eset.up.1103p)
    d.eset.up.1103p = d.eset.up.1103p %>% arrange(gene)
    d.eset.up.1103p['color'] = viridis(length(eset.up.1103p))
    
    pal.ant <- d.eset.up.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P UP ESET')

    g.ver.u.1103p.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P UP ESET')

    g.har.u.1103p.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P UP ESET')
} else {
    print("no 1103p up")
}
eset.down.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]])
if(!is.null(eset.down.1103p)){
    d.eset.down.1103p = data.frame(gene=eset.down.1103p)
    d.eset.down.1103p = d.eset.down.1103p %>% arrange(gene)
    d.eset.down.1103p['color'] = viridis(length(eset.down.1103p))
    
    pal.ant <- d.eset.down.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P DOWN ESET')

    g.ver.u.1103p.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P DOWN ESET')

    g.har.u.1103p.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P DOWN ESET')
} else {
    print("no 1103p down")
}

eset.up.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]])
if(!is.null(eset.up.3309c)){
    d.eset.up.3309c = data.frame(gene=eset.up.3309c)
    d.eset.up.3309c = d.eset.up.3309c %>% arrange(gene)
    d.eset.up.3309c['color'] = viridis(length(eset.up.3309c))
    
    pal.ant <- d.eset.up.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C UP ESET')

    g.ver.u.3309c.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C UP ESET')

    g.har.u.3309c.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C UP ESET')
} else {
    print("no 3309c up")
}
eset.down.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]])
if(!is.null(eset.down.3309c)){
    d.eset.down.3309c = data.frame(gene=eset.down.3309c)
    d.eset.down.3309c = d.eset.down.3309c %>% arrange(gene)
    d.eset.down.3309c['color'] = viridis(length(eset.down.3309c))
    
    pal.ant <- d.eset.down.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C DOWN ESET')

    g.ver.u.3309c.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C DOWN ESET')

    g.har.u.3309c.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C DOWN ESET')
} else {
    print("no 3309c down")
}

eset.up.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]])
if(!is.null(eset.up.so4)){
    d.eset.up.so4 = data.frame(gene=eset.up.so4)
    d.eset.up.so4 = d.eset.up.so4 %>% arrange(gene)
    d.eset.up.so4['color'] = viridis(length(eset.up.so4))
    
    pal.ant <- d.eset.up.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 UP ESET')

    g.ver.u.so4.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 UP ESET')

    g.har.u.so4.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 UP ESET')
} else {
    print("no so4 up")
}
eset.down.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]])
if(!is.null(eset.down.so4)){
    d.eset.down.so4 = data.frame(gene=eset.down.so4)
    d.eset.down.so4 = d.eset.down.so4 %>% arrange(gene)
    d.eset.down.so4['color'] = viridis(length(eset.down.so4))
    
    pal.ant <- d.eset.down.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 DOWN ESET')

    g.ver.u.so4.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 DOWN ESET')

    g.har.u.so4.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 DOWN ESET')
} else {
    print("no so4 down")
}

In [ ]:
# # ran this four times and changed pretty_eset_up, pretty_eset_down, pretty_pset_up, pretty_pset_down
# for (name in names(stats)) {
#     if(grepl("pretty_eset_up", name, fixed=TRUE)){
#         if(grepl("Leaf_2018_Anthesis_Ungrafted", name, fixed=TRUE)){
#             print(name)
#         }
#         if(grepl("Leaf_2018_Veraison_Ungrafted", name, fixed=TRUE)){
#             print(name)
#         }
#         if(grepl("Leaf_2018_Harvest_Ungrafted", name, fixed=TRUE)){
#             print(name)
#         }
#     }
# }